In [4]:
# import the libraries need to be used
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
%matplotlib inline

In [6]:
# Read the data which come from my previous articles
data = pd.read_csv("HK/top3_dist.csv")
data = data.drop({'Unnamed: 0', 'host_id', 'minimum_nights','number_of_reviews', 'calculated_host_listings_count', 'availability_365', 'price'}, axis = 1)
data.head(2)

,id,neighbourhood,latitude,longitude,room_type,cen_lat,cen_long,distance
0,69074,CENTRAL & WESTERN,22.28352,114.15018,Entire home/apt,22.28419,114.146482,0.387495
1,103760,CENTRAL & WESTERN,22.28407,114.15570,Entire home/apt,22.28419,114.146482,0.947967


In this part, the data we would use is located within the YAU TSIM MONG district, so, we have to extract the wanted data:

In [7]:
data_yau = data[data.neighbourhood == 'YAU TSIM MONG']

In [8]:
import folium
from IPython.display import display

Create a data frame of main MTR stations in study area, in this case, we would use Tsuen Wan line:

In [10]:
tsuen_wan_line = {
	'station':['Tsim Sha Tsui Station', 'Jordan Station', 'Yau Ma Tei Station', 'Mong Kok Station', 'Prince Edward Station'],
        'latitude':[22.297560, 22.3048588, 22.313351, 22.319232, 22.324611],
        'longitude':[114.172209, 114.17172, 114.170572, 114.169349, 114.168269]
}
tsuen_wan_line = pd.DataFrame(tsuen_wan_line)
tsuen_wan_line

,station,latitude,longitude
0,Tsim Sha Tsui Station,22.297560,114.172209
1,Jordan Station,22.304859,114.171720
2,Yau Ma Tei Station,22.313351,114.170572
3,Mong Kok Station,22.319232,114.169349
4,Prince Edward Station,22.324611,114.168269


In [11]:
# Convert the coordinates of stations to list
coordinates  = list(zip(tsuen_wan_line.latitude, tsuen_wan_line.longitude))

Define a map centered around Yau Tsim Mong district and plot all data of entire apartment and home, as point features:

In [12]:
hk_map_en = folium.Map(location=[22.31,114.18],zoom_start = 13)
data_yau[data_yau.room_type == 'Entire home/apt'].apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]],radius=1,color='red',fill=False,popup='Entire home/apt',fill_opacity=0.6).add_to(hk_map_en), axis=1)

24      <folium.vector_layers.CircleMarker object at 0...
25      <folium.vector_layers.CircleMarker object at 0...
31      <folium.vector_layers.CircleMarker object at 0...
44      <folium.vector_layers.CircleMarker object at 0...
47      <folium.vector_layers.CircleMarker object at 0...
                              ...                        
9621    <folium.vector_layers.CircleMarker object at 0...
9629    <folium.vector_layers.CircleMarker object at 0...
9632    <folium.vector_layers.CircleMarker object at 0...
9633    <folium.vector_layers.CircleMarker object at 0...
9668    <folium.vector_layers.CircleMarker object at 0...
Length: 1661, dtype: object

And then, draw out the Tsuen Wan Line of MTR:

In [13]:
folium.PolyLine(locations=coordinates,weight=5,color = 'purple').add_to(hk_map_en)
# Plotting the points of the MTR stations in our study area
for lat, lng, label in zip(tsuen_wan_line.latitude, tsuen_wan_line.longitude, tsuen_wan_line.station.astype(str)):
    folium.CircleMarker(
            [lat, lng],
            radius=4,
            color='blue',
            fill=True,
            popup=label,
            fill_color='blue',
            fill_opacity=1
        ).add_to(hk_map_en)

In [14]:
# Finally, save the output as html format:
# hk_map_en.save("map_entire.html")

# Using MarkerCluster to group markers into different clusters

In [15]:
from folium import plugins
# create a map object of Yau Tsim Mong district
hk_map = folium.Map(location=[22.31,114.18],zoom_start = 13)

In [16]:
# And then, drawing out the Tsuen Wan Line of MTR, one of most important MTR route in HK
folium.PolyLine(locations=coordinates,weight=5,color = 'purple').add_to(hk_map)
# Plotting the points of the MTR stations in our study area.
for lat, lng, label in zip(tsuen_wan_line.latitude, tsuen_wan_line.longitude, tsuen_wan_line.station.astype(str)):
    folium.CircleMarker(
            [lat, lng],
            radius=4,
            color='blue',
            fill=True,
            popup=label,
            fill_color='blue',
            fill_opacity=1
        ).add_to(hk_map)

Instatiate a mark cluster object for Aribnb listings:

In [17]:
listings = plugins.MarkerCluster().add_to(hk_map)
# Display the listings inside Yau Tsim Mong
for lat, lng, label, in zip(data_yau.latitude, data_yau.longitude, data_yau.room_type):
    if label != 0:
        folium.Marker(
            location=[lat,lng],
            icon=None,
            popup=label
        ).add_to(listings)

In [18]:
# Save the cluster map to local:
# hk_map.save("map_cluster.html")